
# Generate Readme with up to date list of xRunBooks

## Input

### Import librairies

In [173]:
import os
import json
import requests
import urllib.parse
import copy
from pathlib import Path
import markdown
import nbformat
from nbconvert import MarkdownExporter
from papermill.iorw import (
    load_notebook_node,
    write_ipynb,
)
try:
    from git import Repo
except:
    !pip install GitPython
    from git import Repo

### Variables

In [174]:
# README variables
readme_template = "README_template.md"
readme = "README.md"
replace_var = "[[DYNAMIC_LIST]]"
badge_var = "[[BADGE]]"

# welcome variables
#this is a TODO
#welcome_template = "Welcome_template.ipynb"
#welcome = "Welcome.ipynb"
#replace_var_quote = f'"[[DYNAMIC_LIST]]",\n'


# Others
current_file = '.'
notebook_ext = '.ipynb'
github_url_base = 'https://github.com/unskript/Awesome-CloudOps-Automation/tree/master'
local_OSS_url = 'http://127.0.0.1:8888/lab/tree'
#fix these!
github_download_url = 'https://raw.githubusercontent.com/unskript/Awesome-CloudOps-Automation/master/'
unSkript_logo ='https://unskript.com/assets/favicon.png'


### Get files list

In [175]:
repo = Repo('.')
branch = repo.active_branch
list_of_dir = f"https://api.github.com/repos/unskript/Awesome-CloudOps-Automation/git/trees/{branch.name}?recursive=1"
r_gh = requests.get(list_of_dir).json().get("tree")
notebooks = []
actions=[]
actionCount = 0
runBookCount = 0

main_readme_chart = ""
runbook_list=""
runbook_connector = ""

action_list = ""
action_connector = ""

connector_readme_connector =''
connector_readme = ''
hasRunbook = False
hasAction = False

if r_gh is not None:
    for file in r_gh:
        #only look at files we care about - ignore some of the directories that we don't need to scan
        if ".github" not in file.get("path") and ".gitignore" not in file.get("path") and "templates" not in file.get("path") and "/" in file.get("path")and "__init__" not in file.get("path") and "custom" not in file.get("path")and "unskript-ctl" not in file.get("path"):
            #runbooks are .ipynb, and actions are .py
            if file.get("path").endswith(".ipynb") or file.get("path").endswith(".py"):
                temp = file.get("path").split("/")
                if temp == -1:
                    data = {
                        "root": None,
                        "filename": file.get("path")
                    }
                    notebooks.append(data)
                    
                else:
                    isAction = False
                    last_folder = ""
                    file_name = temp[-1]
                    filename_string = file_name[0:file_name.find(".")]

                    temp.pop()
                    path = ""
                    for folder in temp:
                        path = path + folder +"/"
                        if folder == "legos":
                            isAction = True
                        #print("folder",folder)
                        last_folder += "/" + folder
                    
                    #testing
                    
                    
                    
                    #JSON data
                    filename_json = Path(path +"/"+filename_string+ ".json")
                    jsonData = json.loads(filename_json.read_text())
                    
                    
                    ##we now have a path.. but only really need the root folder
                    ## different ways to generate fior action vs runbook
                    if isAction:
                        actionCount += 1
                        #this is an action folder
                        #find first slash
                        firstslash = last_folder.find("/",1)
                        root = last_folder[1:firstslash]
                        name = jsonData['action_title']
                        description = jsonData['action_description']
                        #categories = jsonData['action_categories']
                        polling = jsonData['action_supports_poll']
                        iteration = jsonData['action_supports_iteration']
                        github_url = github_url_base+"/"+file.get("path")
                        
                    else:
                        runBookCount+=1
                        #root folder for notebooks
                        root = last_folder[1:]
                        name = jsonData['name']
                        description = jsonData['description']
                        categories = jsonData['categories']
                        github_url = github_url_base+"/"+file.get("path")
                    
                    data = {
                        "root": root,
                        "filename": file_name,
                        "name": name,
                        "description": description,
                        "categories":categories,
                        "github_url": github_url
                    }
                    
                    if isAction:
                        data['type'] = "Action"
                        data['polling'] = polling
                        data['iteration']=iteration
                        actions.append(data)
                    else:
                        data['type'] = "RunBook"
                        local_url = local_OSS_url+"/"+file.get("path")
                        data['local_url'] = local_url  
                        notebooks.append(data)
                        
                        
                    #generate the list of runbooks for tha main readme
                    if not isAction:
                        main_readme_chart += f"|{root} |[{name}]({github_url}) | [Open in Browser]({local_url}) | \n"
    
                    #generate the runbook list page
                    if not isAction:
                        #have we created a category yet?
                        if runbook_connector == "":
                            runbook_connector = root
                            runbook_list += f"\n# {runbook_connector}\n"
                        #same category, or new one
                        if runbook_connector != root:
                            # new category
                            runbook_connector = root
                            runbook_list += f"\n# {runbook_connector}\n"
                            
                        #now add in each runbook
                        runbook_list += f"* [{name}]({github_url}): {description}\n"
            
                    #generate the action list page
                    if isAction:
                        #have we created a category yet?
                        if action_connector == "":
                            action_connector = root
                            action_list += f"\n# {action_connector}\n"
                        #same category, or new one
                        if action_connector != root:
                            # new category
                            action_connector = root
                            action_list += f"\n# {action_connector}\n"
                            
                        #now add in each Action
                        action_list += f"* [{name}]({github_url}): {description}\n"
                    
                    

                    #generate the readme for each connector
                    #have we created a category yet?
                    if connector_readme_connector == "":
                        connector_readme_connector = root
                        connector_readme = ''
                        hasRunbook = False
                        hasAction = False
                    #same category, or new one
                    if connector_readme_connector != root:
                        # starting a new readme
                        #first let's save the old one:
                        #print(connector_readme)
                        readme_file = f"{connector_readme_connector}/README.md"
                        f  = open(readme_file, "w+")
                        f.write(connector_readme)
                        f.close()
                        #now start building the new readme
                        connector_readme_connector = root
                        connector_readme = ''
                        hasRunbook = False
                        hasAction = False

                    if data['type'] =="RunBook":
                        if not hasRunbook:
                            connector_readme += f'# {root} RunBooks\n'
                            hasRunbook = True
                        connector_readme += f"* [{name}]({github_url}): {description}\n"
                    if data['type'] =="Action":
                        if not hasAction:
                            connector_readme += f'\n# {root} Actions\n'
                            hasAction = True
                        connector_readme += f"* [{name}]({github_url}): {description}\n"
    
#print(notebooks)
#print(action_list)
#print(runBookCount, actionCount)


In [176]:
## generate category lists from runbooks

notebook_categories = {}

for notebook in notebooks:
    #print(notebook['root'], notebook['name'])
    if len(notebook['categories'])>0:
        #print(notebook['categories'])
        for category in notebook['categories']:
            if  category in notebook_categories:
                notebook_categories[category].append(notebook)
            else:
                notebook_categories[category] = []

category_name =""
category_listing = ""
for category in notebook_categories:
    if category_name =="":
        #new category
        category_name = category
        category_listing = f"# Runbooks in {category}\n"
    if category_name != category:
        # we have finished off a category
        #save the oldcategory
        print(category_name, category_listing)
        categoryList_filename = Path(f"lists/runbook_{category_name}.md")
        f  = open(categoryList_filename, "w+")
        f.write(category_listing)
        f.close()
        category_name = category
        category_listing = f"# Runbooks in {category}\n"
    

    for runbook in notebook_categories[category]:
        category_listing += f"*{runbook['root']} [{runbook['name']}]({runbook['github_url']}): {runbook['description']}\n"
        


CATEGORY_TYPE_IAM # Runbooks in CATEGORY_TYPE_IAM
*AWS [Create a new AWS IAM User](https://github.com/unskript/Awesome-CloudOps-Automation/tree/master/AWS/Add_new_IAM_user.ipynb): AWS has an inbuilt identity and access management system known as AWS IAM. IAM supports the concept of users, group, roles and privileges. IAM user is an identity that can be created and assigned some privileges. This runbook can be used to create an AWS IAM User
*AWS [Update and Manage AWS User permission](https://github.com/unskript/Awesome-CloudOps-Automation/tree/master/AWS/Update_and_Manage_AWS_User_Permission.ipynb): This runbook can be used Update and Manage AWS IAM User Permission

CATEGORY_TYPE_SECOPS # Runbooks in CATEGORY_TYPE_SECOPS
*AWS [Create a new AWS IAM User](https://github.com/unskript/Awesome-CloudOps-Automation/tree/master/AWS/Add_new_IAM_user.ipynb): AWS has an inbuilt identity and access management system known as AWS IAM. IAM supports the concept of users, group, roles and privileges. 

### Preview the generated list

### Generate readme for github repository

In [177]:
# Open README template
template = open(readme_template).read()

# Replace var to get list of templates in markdown format
template = template.replace(replace_var, main_readme_chart)

#create the action and runbook badges
actionImage = f'https://img.shields.io/static/v1?label=ActionCount&message={actionCount}&color=green'
runbookImage = f'https://img.shields.io/static/v1?label=xRunBookCount&message={runBookCount}&color=orange'
actionbadge = f"<img src={actionImage}>"
runbookbadge =  f"<img src={runbookImage}>"


#insert the badges on the readme
badgeCode = actionbadge + runbookbadge
template =  template.replace(badge_var, badgeCode)

# Save README
f  = open(readme, "w+")
f.write(template)
f.close()

In [178]:

#generate a page of all the actions that are available
ac_list= readme = "lists/Action_list.md"
# Save README
f  = open(ac_list, "w+")
f.write(action_list)
f.close()


In [179]:
#generate a page of all the Runbooks that are available
rb_list = readme = "lists/xRunBook_list.md"
# Save README
f  = open(rb_list, "w+")
f.write(runbook_list)
f.close()

In [180]:
#create JSON files that can be used t build badges for the website showing our Runbook and action counts

import json
#store Runbookcount json (to create shield on webpage)
rb_list = readme = ".github/images/runbookShield.json"
json1 = {"schemaVersion": 1,"label": "RunBook Count","message": runBookCount,"color": "orange"}

# Save README

with open(rb_list, "w") as outfile:
    json.dump(json1, outfile)


#store action count json (to create shield on webpage)
rb_list = readme = ".github/images/actionShield.json"
json2 = {"schemaVersion": 1,"label": "Action Count","message": actionCount,"color": "green"}

# Save README
with open(rb_list, "w") as outfile:
    json.dump(json2, outfile)